In [ ]:
import os
import random
import numpy as np
import pandas as pd
import joblib
import json
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    roc_auc_score, log_loss, brier_score_loss,
    precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_error, r2_score
)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def set_random_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ['PYTHONHASHSEED'] = "42"

In [ ]:
set_random_seeds(42)

In [ ]:
# features excluded from the final feature set based on prior feature selection
# + technical columns
excluded_features = [ 'x','sq_flag',
    'drct_v_lag_6h','drct_v_lag_12h','drct_v_lag_24h','drct_v_lag_48h',
    'drct_u_lag_6h','drct_u_lag_24h','drct_u_lag_48h',
    'tmpf_lag_12h','tmpf_lag_24h','tmpf_lag_48h',
    'hr_flag_rolling_sum_6h','hr_flag_rolling_sum_12h','hr_flag_rolling_sum_24h','hr_flag_rolling_sum_48h',
    'sq_flag_rolling_sum_6h','sq_flag_rolling_sum_12h','sq_flag_rolling_sum_24h','sq_flag_rolling_sum_48h',
    'relh_grad_rolling_max_6h','relh_grad_rolling_max_12h','relh_grad_rolling_max_24h','relh_grad_rolling_max_48h',
    'ts_flag_rolling_sum_6h','ts_flag_rolling_sum_12h','ts_flag_rolling_sum_24h','ts_flag_rolling_sum_48h',
    'mslp_rolling_mean_6h','mslp_rolling_mean_12h','mslp_rolling_mean_24h','mslp_rolling_mean_48h',
    'gust_rolling_max_12h','gust_rolling_max_24h','gust_rolling_max_48h',
    'alti_rolling_mean_6h','alti_rolling_mean_12h','alti_rolling_mean_24h','alti_rolling_mean_48h',
    'sknt_rolling_max_6h','sknt_rolling_max_12h','sknt_rolling_max_24h','sknt_rolling_max_48h',
    'relh_rolling_mean_6h','relh_rolling_mean_12h','relh_rolling_mean_24h','relh_rolling_mean_48h',
    'p0li_rolling_sum_6h','p0li_rolling_sum_12h','p0li_rolling_sum_24h','p0li_rolling_sum_48h',
    'time_interval','anomaly_threshold', 'year',
    'IDW_ts_flag_rolling_sum_12h']

df = df.drop(columns=excluded_features, errors='ignore')

Undersampling

In [ ]:
# train and test split
df = df.sort_values('run_start_time').reset_index(drop=True)

df_train = df[df.run_start_time.dt.year == 2021].copy()
df_test  = df[df.run_start_time.dt.year == 2022].copy()

print("Train shape (before):", df_train.shape)
print(df_train['target_anomaly_48h'].value_counts())

# prepare 48h windows
df_train['time_interval'] = df_train['run_start_time'].dt.floor('48H')

kept_intervals = []

# iteration on counties
for county, df_county in df_train.groupby('fips_code'):

    for interval, interval_data in df_county.groupby('time_interval'):
        interval_data = interval_data.copy()

        y = interval_data['target_anomaly_48h'].values
        num_pos, num_neg = y.sum(), len(y) - y.sum()

        if (num_pos >= 3) and (num_neg >= 1):     # append windows if the number of Class 1 >= 3 and Class 0 >= 1 to avoid uninformative windows
            kept_intervals.append(interval_data)

# concatenate resulted windows
df_train_us = (                       # us = undersampled
    pd.concat(kept_intervals, ignore_index=True)
      .sort_values(['run_start_time', 'fips_code'])
      .reset_index(drop=True)
)

print("\nTrain shape (after):", df_train_us.shape)
print(df_train_us['target_anomaly_48h'].value_counts())

print("\nTest shape :", df_test.shape)
print(df_test['target_anomaly_48h'].value_counts())

print('Train period:', df_train_us.run_start_time.min(), '–', df_train_us.run_start_time.max())
print('Test period :', df_test.run_start_time.min(),  '–', df_test.run_start_time.max())

1. Logistic Regression

In [ ]:
# Stage-1 Cross Fold Validation

features_to_drop = [
    'run_start_time', 'target_anomaly_48h', 'target_flag', 'sum_48','fips_code', # technical columns
    'state_sum_48h', 'target_flag_24','run_start_time_plus_48','sum','anomaly_flag',

    'dwpf','tmpf','relh','alti','mslp','gust','p0li','sknt','drct_u','drct_v','relh_grad','ts_flag','hr_flag', # LSTM features
    'dwpf_lag_6h','dwpf_lag_24h','dwpf_lag_48h','gust_rolling_max_6h','idw_rolling_max_24h','IDW_dwpf','IDW_alti',
    'IDW_drct_v','IDW_drct_u','IDW_drct_v_lag_6h','IDW_p0li_rolling_sum_24h','IDW_gust_rolling_max_24h',
    'IDW_sknt_rolling_max_48h','IDW_relh_rolling_mean_48h',
    'day_of_week_num','population_density','county_encoded','y'

]

# settings
threshold   = 0.50
TARGET_COL  = 'target_anomaly_48h'
N_SPLITS    = 3
C_VAL       = 0.001
CLASS_WT    = {0: 1, 1: 5}

X_us = df_train_us.drop(columns=features_to_drop, errors='ignore').copy()
y_us = df_train_us[TARGET_COL].copy()

# keep numeric columns only
non_num = X_us.select_dtypes(exclude=['int64', 'float64']).columns
if len(non_num):
    X_us.drop(columns=non_num, inplace=True)

# replace infinite values with NaN for imputer
X_us.replace([np.inf, -np.inf], np.nan, inplace=True)

num_cols = X_us.columns          # final feature set

# time based split
times     = np.sort(df_train_us['run_start_time'].unique())
fold_len  = len(times) // (N_SPLITS + 1)
folds     = [(times[: i * fold_len],
              times[i * fold_len : (i + 1) * fold_len])
             for i in range(1, N_SPLITS + 1)]


metrics = {'train': [], 'val': []}
coefs   = []

# scaling
imp    = SimpleImputer(strategy='median') # median imputer
scaler = MinMaxScaler()

for k, (t_train, t_val) in enumerate(folds, 1):
    idx_tr = df_train_us['run_start_time'].isin(t_train)
    idx_va = df_train_us['run_start_time'].isin(t_val)

    X_tr = X_us.loc[idx_tr, num_cols].copy()
    y_tr = y_us[idx_tr].copy()
    X_va = X_us.loc[idx_va, num_cols].copy()
    y_va = y_us[idx_va].copy()

    # imputer and scaling
    X_tr[num_cols] = imp.fit_transform(X_tr[num_cols])
    X_va[num_cols] = imp.transform(X_va[num_cols])
    X_tr[num_cols] = scaler.fit_transform(X_tr[num_cols])
    X_va[num_cols] = scaler.transform(X_va[num_cols])

    # model
    lg = LogisticRegression(C=C_VAL, class_weight=CLASS_WT,
                            penalty='l2', solver='lbfgs',
                            max_iter=1000, random_state=42)
    lg.fit(X_tr, y_tr)
    coefs.append(lg.coef_[0])

    # metrics
    for tag, X_, y_ in [('train', X_tr, y_tr), ('val', X_va, y_va)]:
        p = lg.predict_proba(X_)[:, 1]
        m = {
            'AUC'      : roc_auc_score(y_, p),
            'LogLoss'  : log_loss(y_, p),
            'Brier'    : brier_score_loss(y_, p),
            'Precision': precision_score(y_, p >= threshold, zero_division=0),
            'Recall'   : recall_score(   y_, p >= threshold, zero_division=0),
            'F1'       : f1_score(       y_, p >= threshold, zero_division=0)
        }
        metrics[tag].append(m)

    tr = metrics['train'][-1]; va = metrics['val'][-1]
    print(f"\nFold {k}")
    print("  TRAIN:", ", ".join(f"{k}:{v:.3f}" for k,v in tr.items()))
    print("  VAL  :", ", ".join(f"{k}:{v:.3f}" for k,v in va.items()))

# average on folds
avg = lambda lst: {m: np.mean([d[m] for d in lst]) for m in lst[0]}

print("\n=== Average metrics over folds ===")
print("TRAIN:", ", ".join(f"{k}:{v:.3f}" for k,v in avg(metrics['train']).items()))
print("VAL  :", ", ".join(f"{k}:{v:.3f}" for k,v in avg(metrics['val']).items()))

# feature importance
avg_coef = np.mean(coefs, axis=0)
imp_df = (pd.DataFrame({'feature': num_cols,
                        'coef': avg_coef,
                        'abs': np.abs(avg_coef)})
            .sort_values('abs', ascending=False))

print("\nTop-15 features (|coef|):")
print(imp_df.head(15))

In [ ]:
# Stage-1: Final train on the entire df_train_us with the same imputer and a new scaler_full

# train imputer on full df_train_us
imp_full = SimpleImputer(strategy='median')
imp_full.fit(X_us[num_cols])

X_imp_full = imp_full.transform(X_us[num_cols])

# train scaler
scaler_full = MinMaxScaler()
scaler_full.fit(X_imp_full)

X_scaled_full = scaler_full.transform(X_imp_full)

# final model
final_model = LogisticRegression(
        C=0.001, penalty='l2', class_weight={0: 1, 1: 5},
        solver='lbfgs', max_iter=1000, random_state=42
)
final_model.fit(X_scaled_full, y_us)

print("\n✔  Stage-1 final model trained on **all** df_train_us")

# metrics
proba_full = final_model.predict_proba(X_scaled_full)[:, 1]
pred_full  = (proba_full >= threshold).astype(int)

print("\n===  Metrics on FULL df_train_us  ===")
print(f"AUC      : {roc_auc_score(y_us, proba_full):.3f}")
print(f"LogLoss  : {log_loss(y_us, proba_full):.3f}")
print(f"Brier    : {brier_score_loss(y_us, proba_full):.3f}")
print(f"Prec / Rec / F1 = "
      f"{precision_score(y_us, pred_full):.3f} / "
      f"{recall_score(  y_us, pred_full):.3f} / "
      f"{f1_score(      y_us, pred_full):.3f}")

In [ ]:
# Stage-1: train on the real train set
threshold   = 0.7 # updated thershold for real imbalanced training set
TARGET_COL  = 'target_anomaly_48h'


df_train_real = df_train.copy()

X_train_real = (
    df_train_real
      .drop(columns=features_to_drop, errors='ignore')
      .copy()
)

# train global imputer and scaler
imp = SimpleImputer(strategy='median')
imp.fit(X_us[num_cols])

X_imp = pd.DataFrame(
    imp.transform(X_us[num_cols]),
    columns=num_cols,
    index=X_us.index
)

scaler_full = MinMaxScaler()
scaler_full.fit(X_imp)

cols_imp    = list(imp.feature_names_in_)   # features after imputer
cols_scaler = cols_imp[:]                   # the same order for MinMax


# reindex infinitive values and NaN for imputer
X_train_real = X_train_real.reindex(columns=cols_imp)
X_train_real.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train_real.loc[:, :] = imp.transform(X_train_real)


X_train_scaled = pd.DataFrame(
    X_train_real[cols_scaler].values,
    columns=cols_scaler,
    index=X_train_real.index
)
X_train_scaled.loc[:, :] = scaler_full.transform(X_train_scaled)

# prediction and metrics
y_train_proba = final_model.predict_proba(X_train_scaled)[:, 1]
df_train_real['pred_class_1stage'] = (y_train_proba >= threshold).astype(int)

y_true = df_train_real[TARGET_COL].values
y_pred = df_train_real['pred_class_1stage'].values

print("\n=== Metrics on REAL TRAIN (summer-2021) ===")
print(f"AUC     : {roc_auc_score(y_true, y_train_proba):.3f}")
print(f"LogLoss : {log_loss(y_true, y_train_proba):.3f}")
print(f"Brier   : {brier_score_loss(y_true, y_train_proba):.3f}")
print(f"Precision / Recall / F1 = "
      f"{precision_score(y_true, y_pred, zero_division=0):.3f} / "
      f"{recall_score(   y_true, y_pred, zero_division=0):.3f} / "
      f"{f1_score(       y_true, y_pred, zero_division=0):.3f}")

In [ ]:
# Stage-1: train on Test set
threshold  = 0.7          # the same thershold as used on full train
TARGET_COL = 'target_anomaly_48h'

df_test_real = df_test.copy()

# feature set
X_test_real = (
    df_test_real
      .drop(columns=features_to_drop, errors='ignore')
      .copy()
)


# the same column list
cols_imp    = list(imp.feature_names_in_)             # for SimpleImputer
cols_scaler = list(scaler_full.feature_names_in_)     # for MinMaxScaler

# imputed set
X_test_real = X_test_real.reindex(columns=cols_imp)

# use NaN for infinitive value before imputation
X_test_real.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test_real.loc[:, :] = imp.transform(X_test_real)


# keep scaler columns
X_test_scaled = pd.DataFrame(
    X_test_real[cols_scaler].values,
    columns=cols_scaler,
    index=X_test_real.index
)
X_test_scaled.loc[:, :] = scaler_full.transform(X_test_scaled)

# probability of class 1
y_test_proba = final_model.predict_proba(X_test_scaled)[:, 1]
df_test_real['pred_class_1stage'] = (y_test_proba >= threshold).astype(int)
# df_test_real['pred_proba_1stage'] = y_test_proba   # if needed

# metrics
from sklearn.metrics import (roc_auc_score, log_loss, brier_score_loss,
                             precision_score, recall_score, f1_score)

y_true = df_test_real[TARGET_COL].values
y_pred = df_test_real['pred_class_1stage'].values

print("\n=== Metrics on TEST-2022 ===")
print(f"AUC     : {roc_auc_score(y_true, y_test_proba):.3f}")
print(f"LogLoss : {log_loss(y_true, y_test_proba):.3f}")
print(f"Brier   : {brier_score_loss(y_true, y_test_proba):.3f}")
print(f"P / R / F1 = "
      f"{precision_score(y_true, y_pred, zero_division=0):.3f} / "
      f"{recall_score(   y_true, y_pred, zero_division=0):.3f} / "
      f"{f1_score(       y_true, y_pred, zero_division=0):.3f}")

In [ ]:

TARGET_COL = 'target_anomaly_48h'
PRED_COL   = 'pred_class_1stage'      # classification label

def audit_stage1(df: pd.DataFrame, tag: str = "TRAIN"):
    """ Statistics after Stage-1."""""

    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(df[TARGET_COL], df[PRED_COL]).ravel()
    print(f"\n=== {tag} — confusion matrix ===")
    print(f"TP = {tp:>6}   FP = {fp:>6}")
    print(f"FN = {fn:>6}   TN = {tn:>6}")

    # Class balance before Stage-1
    share_pos_raw = df[TARGET_COL].mean()
    print(f"\nRaw class balance   :  pos = {share_pos_raw:6.2%}   "
          f"neg = {1-share_pos_raw:6.2%}")

    # Class balance after Stage-1 (Stage-2 sample set)
    df_stage2 = df.query(f"{PRED_COL} == 1")
    share_pos_stage2 = df_stage2[TARGET_COL].mean() if not df_stage2.empty else 0
    print(f"After Stage-1 filter:  pos = {share_pos_stage2:6.2%}   "
          f"neg = {1-share_pos_stage2:6.2%}")
    print(f"Stage-2 sample size :  {len(df_stage2):,} rows "
          f"(из {len(df):,},  {len(df_stage2)/len(df):.2%})")


audit_stage1(df_train_real, "TRAIN-2021")
audit_stage1(df_test_real , "TEST-2022")

2. LSTM

In [ ]:
# Stage-2: LSTM Regressor
# uses the same dataset as in Stage-1 but training only on samples where Stage-1 predicted class == 1

# feature names as seen by SimpleImputer and MinMaxScaler during fit
cols_imp    = list(imp.feature_names_in_) # used for imputer
cols_scaler = list(scaler_full.feature_names_in_) # used for scaling

# kep only Stage-1 sample == class 1
df_reg_train_stage2 = df_train_real.query("pred_class_1stage == 1").copy()
df_reg_test_stage2  = df_test_real .query("pred_class_1stage == 1").copy()

for _df in (df_reg_train_stage2, df_reg_test_stage2):

    # drop unused features and reindex to match imputer inputation
    _df_feats = (_df
                 .drop(columns=features_to_drop, errors='ignore')
                 .reindex(columns=cols_imp))

    # replace infinitive values with NaN and apply imputation
    _df_feats.replace([np.inf, -np.inf], np.nan, inplace=True)
    _df_feats.loc[:, :] = imp.transform(_df_feats)

    # applu scaling using Stage-1 scaler
    _df_scaled = pd.DataFrame(
        _df_feats[cols_scaler].values,
        columns=cols_scaler,
        index=_df_feats.index
    )
    _df_scaled.loc[:, :] = scaler_full.transform(_df_scaled)

    # write scaled features back to the dataframe
    _df.loc[:, cols_scaler] = _df_scaled

    # Add log transformed regression target
    _df['log_sum_48'] = np.log1p(_df['sum_48'])

print("Reg train stage2:", df_reg_train_stage2.shape)
print("Reg  test stage2:", df_reg_test_stage2.shape)

In [ ]:
# Stage-2: Cross Validation

# X and Y
df_reg_2021 = df_reg_train_stage2.copy()

# columns to drop
cols_to_drop = [
    'pred_class_1stage', 'pred_class_1stage_proba',                               # technical columns
    'run_start_time', 'target_anomaly_48h', 'target_flag', 'sum_48','fips_code',
    'state_sum_48h', 'target_flag_24','run_start_time_plus_48','sum','anomaly_flag',
    'log_sum_48',

    'rolling_max_12h','rolling_mean_12h','tmpf_lag_6h','drct_u_lag_12h' # Stage-1 features
]

X_2021 = df_reg_2021.drop(columns=cols_to_drop, errors='ignore')
y_2021 = df_reg_2021['log_sum_48'].copy()

# drop non numeric columns
non_numeric_cols = X_2021.select_dtypes(exclude=['int64','float64']).columns
if len(non_numeric_cols) > 0:
    print("Dropping non-numeric columns from X_2021:", non_numeric_cols.tolist())
    X_2021.drop(columns=non_numeric_cols, inplace=True, errors='ignore')

print("X_2021 shape:", X_2021.shape, "y_2021 shape:", y_2021.shape)

# Timeseries split wirh 3 folds
tscv = TimeSeriesSplit(n_splits=3)

fold_metrics = []

numeric_cols = X_2021.columns

# Iteration by folds
for fold_idx, (train_index, val_index) in enumerate(tscv.split(X_2021), start=1):
    # split folds
    X_train_fold = X_2021.iloc[train_index].copy()
    y_train_fold = y_2021.iloc[train_index].copy()

    X_val_fold   = X_2021.iloc[val_index].copy()
    y_val_fold   = y_2021.iloc[val_index].copy()

    # infinitive to NaN
    for _df in (X_train_fold, X_val_fold):
        _df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # imputer
    imp_fold = SimpleImputer(strategy='median')
    X_train_fold[numeric_cols] = imp_fold.fit_transform(X_train_fold[numeric_cols])
    X_val_fold[numeric_cols]   = imp_fold.transform(   X_val_fold[numeric_cols])

    scaler_fold = MinMaxScaler()
    X_train_fold[numeric_cols] = scaler_fold.fit_transform(X_train_fold[numeric_cols])
    X_val_fold[numeric_cols]   = scaler_fold.transform(   X_val_fold[numeric_cols])


    # Приводим к (samples, 1, features)
    X_train_3d = X_train_fold.values.reshape(
        (X_train_fold.shape[0], 1, X_train_fold.shape[1])
    )
    X_val_3d   = X_val_fold.values.reshape(
        (X_val_fold.shape[0], 1, X_val_fold.shape[1])
    )

    # model
    n_features = X_train_3d.shape[2]
    model = keras.Sequential()
    model.add(layers.LSTM(16, input_shape=(1, n_features)))
    model.add(layers.Dense(1, activation='relu'))
    model.compile(optimizer='adam', loss='mse')


    early_stopping = EarlyStopping(
       monitor='val_loss', patience=2, restore_best_weights=True
    )

    # fit
    model.fit(
        X_train_3d, y_train_fold,
        epochs=30, batch_size=32,
        validation_data=(X_val_3d, y_val_fold),
        callbacks=[early_stopping],
        verbose=1
    )

    # predict
    y_val_pred = model.predict(X_val_3d).ravel()

    # Log metrics
    mse_val_log = mean_squared_error(y_val_fold, y_val_pred)
    rmse_val_log = np.sqrt(mse_val_log)
    mae_val_log = mean_absolute_error(y_val_fold, y_val_pred)
    r2_val_log = r2_score(y_val_fold, y_val_pred)

    # Metrics in original scale
    y_val_true_orig = np.expm1(y_val_fold)
    y_val_pred_orig = np.expm1(y_val_pred)
    mse_val_orig = mean_squared_error(y_val_true_orig, y_val_pred_orig)
    rmse_val_orig = np.sqrt(mse_val_orig)
    mae_val_orig = mean_absolute_error(y_val_true_orig, y_val_pred_orig)
    r2_val_orig = r2_score(y_val_true_orig, y_val_pred_orig)

    # all metrics in one dict
    fold_metrics.append({
       'mse_log':  mse_val_log,
       'rmse_log': rmse_val_log,
       'mae_log':  mae_val_log,
       'r2_log':   r2_val_log,
       'mse_orig':  mse_val_orig,
       'rmse_orig': rmse_val_orig,
       'mae_orig':  mae_val_orig,
       'r2_orig':   r2_val_orig
    })

    print(f"\n[Fold {fold_idx} / {tscv.n_splits}] LOG-scale => "
          f"MSE={mse_val_log:.4f}, RMSE={rmse_val_log:.4f}, MAE={mae_val_log:.4f}, R^2={r2_val_log:.4f}")
    print(f"                       Orig-scale => "
          f"MSE={mse_val_orig:.2f}, RMSE={rmse_val_orig:.2f}, MAE={mae_val_orig:.2f}, R^2={r2_val_orig:.4f}")

# averaged metrics by folds
def mean_of_metric(fold_metrics, key):
    return np.mean([fm[key] for fm in fold_metrics])

avg_mse_log  = mean_of_metric(fold_metrics, 'mse_log')
avg_rmse_log = mean_of_metric(fold_metrics, 'rmse_log')
avg_mae_log  = mean_of_metric(fold_metrics, 'mae_log')
avg_r2_log   = mean_of_metric(fold_metrics, 'r2_log')

print("\n=== Cross-Validation (log-scale) ===")
print(f"MSE:  {avg_mse_log:.4f}")
print(f"RMSE: {avg_rmse_log:.4f}")
print(f"MAE:  {avg_mae_log:.4f}")
print(f"R^2:  {avg_r2_log:.4f}")

avg_mse_orig  = mean_of_metric(fold_metrics, 'mse_orig')
avg_rmse_orig = mean_of_metric(fold_metrics, 'rmse_orig')
avg_mae_orig  = mean_of_metric(fold_metrics, 'mae_orig')
avg_r2_orig   = mean_of_metric(fold_metrics, 'r2_orig')

print("\n=== Cross-Validation (original scale) ===")
print(f"MSE:  {avg_mse_orig:.2f}")
print(f"RMSE: {avg_rmse_orig:.2f}")
print(f"MAE:  {avg_mae_orig:.2f}")
print(f"R^2:  {avg_r2_orig:.4f}")

In [ ]:
# Stage-2 Train (2021)

print("Initial train shapes :", X_2021.shape, y_2021.shape)

# list of numeric columns
num_cols = X_2021.columns

# median-impute
X_2021.replace([np.inf, -np.inf], np.nan, inplace=True)
imp_final = SimpleImputer(strategy='median')
X_2021[num_cols] = imp_final.fit_transform(X_2021[num_cols])

# Min–Max scailing
scaler_final = MinMaxScaler()
X_2021_scaled = scaler_final.fit_transform(X_2021[num_cols])

# reshape (samples, 1, n_features)
X_2021_3d = X_2021_scaled.reshape(
    (X_2021_scaled.shape[0], 1, X_2021_scaled.shape[1])
)
print("LSTM input shapes   :", X_2021_3d.shape, y_2021.shape)

# LSTM Model (one-step)
n_features = X_2021_3d.shape[2]
model_final = keras.Sequential([
    layers.LSTM(16, input_shape=(1, n_features)),
    layers.Dense(1, activation='relu')
])
model_final.compile(optimizer='adam', loss='mse')

early_stopping = EarlyStopping(
    monitor='loss', patience=2, restore_best_weights=True
)

model_final.fit(
    X_2021_3d, y_2021,
    epochs=30, batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)


# Metrics on Training set (log-scale)
y_pred_log = model_final.predict(X_2021_3d, verbose=0).ravel()

mse_log  = mean_squared_error(y_2021, y_pred_log)
rmse_log = np.sqrt(mse_log)
mae_log  = mean_absolute_error(y_2021, y_pred_log)
r2_log   = r2_score(y_2021, y_pred_log)

print("\nFinal model TRAIN (log-scale)")
print(f"MSE={mse_log:.4f}  RMSE={rmse_log:.4f}  "
      f"MAE={mae_log:.4f}  R²={r2_log:.4f}")

# Metrics on Training set (original scale)

y_true_orig = np.expm1(y_2021)
y_pred_orig = np.expm1(y_pred_log)

mse_orig  = mean_squared_error(y_true_orig, y_pred_orig)
rmse_orig = np.sqrt(mse_orig)
mae_orig  = mean_absolute_error(y_true_orig, y_pred_orig)
r2_orig   = r2_score(y_true_orig, y_pred_orig)

print("\nFinal model TRAIN (original scale) ")
print(f"MSE={mse_orig:,.2f}  RMSE={rmse_orig:,.2f}  "
      f"MAE={mae_orig:,.2f}  R²={r2_orig:.4f}")

# save imputer for test set
joblib.dump(imp_final,    "imp_stage2.joblib")
joblib.dump(scaler_final, "scaler_stage2.joblib")

In [ ]:
# Stage-2 Test (2022)

df_reg_test_2022 = df_reg_test_stage2.copy()

# numeric columns
X_test = (
    df_reg_test_2022
      .drop(columns=cols_to_drop, errors='ignore')
      .reindex(columns=num_cols)          # fixing the order
)
y_test_log = df_reg_test_2022['log_sum_48'].copy()

# infinitive to NaN; SimpleImputer; scaler_final
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test[num_cols] = imp_final.transform(X_test[num_cols])     # using the same imputer
X_test_scaled    = scaler_final.transform(X_test[num_cols])  # only .transform

# reshape for LSTM
X_test_3d = X_test_scaled.reshape(
    (X_test_scaled.shape[0], 1, X_test_scaled.shape[1])
)

# predict
y_test_pred_log = model_final.predict(X_test_3d).ravel()

# metrics (log-scale)
mse_log  = mean_squared_error(y_test_log, y_test_pred_log)
rmse_log = np.sqrt(mse_log)
mae_log  = mean_absolute_error(y_test_log, y_test_pred_log)
r2_log   = r2_score(y_test_log, y_test_pred_log)

print("\nFinal model on TEST-2022 (log-scale)")
print(f"MSE={mse_log:.4f}, RMSE={rmse_log:.4f}, MAE={mae_log:.4f}, R²={r2_log:.4f}")

# metrics (original scale)
y_true_orig = np.expm1(y_test_log)
y_pred_orig = np.expm1(y_test_pred_log)

mse_orig  = mean_squared_error(y_true_orig, y_pred_orig)
rmse_orig = np.sqrt(mse_orig)
mae_orig  = mean_absolute_error(y_true_orig, y_pred_orig)
r2_orig   = r2_score(y_true_orig, y_pred_orig)

print("\nFinal model on TEST-2022 (original scale)")
print(f"MSE={mse_orig:.2f}, RMSE={rmse_orig:.2f}, MAE={mae_orig:.2f}, R²={r2_orig:.4f}")